In [13]:
import ASG
import Utility
import os
# import importlib
# importlib.reload(Utility)

In [30]:
# Only need to change the path variable
# path = "../../C ASG statistics 1115ver/saved_pkl/Dofloo/0046a78514a658b9b7a4e29e8581f85b.bin"
path = "../../C ASG/trace/Dofloo/0046a78514a658b9b7a4e29e8581f85b.bin"
print("path is correct:", os.path.exists(path))
graph = ASG.AttackGraph(path)
graph.create()
graph.step_list[:3]

path is correct: True


[(<ASG.Node at 0x24f45f6a7d0>, <ASG.Node at 0x24f25be3550>, 'read'),
 (<ASG.Node at 0x24f25be3550>, <ASG.Node at 0x24f45f6ae30>, 'brk'),
 (<ASG.Node at 0x24f25be3550>, <ASG.Node at 0x24f45f6ae30>, 'brk')]

# For file regex

In [31]:
set_of_objects_file = Utility.get_set_of_objects_file(graph)

# rule dict from .xlsx
RULES_DICT = Utility.build_RULES_DICT() 

# use the rule dict to build up the regex, it may not match all objects
regex_match_file, regex_non_match_file = Utility.build_file_regex(set_of_objects_file, RULES_DICT) 
file_regex = list(regex_match_file.keys())

print("Objects num:", len(set_of_objects_file))
print("Match Regex num:", len(regex_match_file))
print("Non match file num:", len(regex_non_match_file))

Objects num: 21
Match Regex num: 12
Non match file num: 0


In [32]:
file_regex

['/.*bin/sed',
 '/dev/urandom',
 '/etc/init\\.d/.*',
 '/etc/rc.*\\.d/.*',
 '/etc/rc\\.local',
 '/etc/sed[0-9a-zA-Z]{6}',
 '.*/selinux',
 '/proc/net/dev',
 '/proc/self',
 '/proc/stat',
 '/sys/devices/.*',
 'uname']

In [33]:
regex_match_file

{'/.*bin/sed': ['/bin/sed',
  '/sbin/sed',
  '/usr/bin/sed',
  '/usr/local/bin/sed',
  '/usr/local/sbin/sed',
  '/usr/sbin/sed'],
 '/dev/urandom': ['/dev/urandom'],
 '/etc/init\\.d/.*': ['/etc/init.d/boot.local'],
 '/etc/rc.*\\.d/.*': ['/etc/rc.d/rc.local'],
 '/etc/rc\\.local': ['/etc/rc.local'],
 '/etc/sed[0-9a-zA-Z]{6}': ['/etc/sedQUGLbs',
  '/etc/sedQhw17q',
  '/etc/sedvTqQwq'],
 '.*/selinux': ['/sys/fs/selinux'],
 '/proc/net/dev': ['/proc/net/dev'],
 '/proc/self': ['/proc/self/exe'],
 '/proc/stat': ['/proc/stat'],
 '/sys/devices/.*': ['/sys/devices/system/cpu/online'],
 'uname': ['uname']}

In [34]:
regex_non_match_file

[]

# For remain regex

In [35]:
proc_regex = Utility.get_proc_regex(graph)
net_regex = Utility.get_net_regex(graph)
mem_regex = Utility.get_mem_regex(graph)
ID_regex = Utility.get_ID_regex(graph)
permission_regex = Utility.get_premission_regex(graph)

In [36]:
proc_regex

['sh', 'sed']

In [37]:
net_regex

['eth.*',
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}',
 'port d+ ',
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:d+']

In [38]:
mem_regex

['0x[0-9a-zA-Z]{8}']

In [39]:
ID_regex

['UID.*', 'GID.*', 'PID.*']

In [40]:
permission_regex

['Permission', 'Permission:[0-9]{3}']

# check all object

In [41]:
# This contain all object including "subject" and "object"
Utility.get_set_of_dict(graph)

{'File': ['uname',
  '/proc/self/exe',
  '/prober',
  '.',
  '/usr/local/sbin/sed',
  '/usr/local/bin/sed',
  '/usr/sbin/sed',
  '/usr/bin/sed',
  '/sbin/sed',
  '/bin/sed',
  '/sys/fs/selinux',
  '/selinux',
  '/etc/selinux/config',
  '/etc/rc.local',
  '/etc/sedQUGLbs',
  '/etc/sedQhw17q',
  '/etc/sedvTqQwq',
  '/etc/rc.d/rc.local',
  '/etc/init.d/boot.local',
  '/dev/urandom',
  '/proc/net/dev',
  '/sys/devices/system/cpu/online',
  '/proc/stat'],
 'Process': ['sh', '1526', 'sed', 'NO_PID', '1527', '1528', '1529'],
 'Net': ['eth0',
  'eth1',
  'eth2',
  'eth3',
  'eth4',
  'eth5',
  'eth6',
  'eth7',
  'eth8',
  'eth9',
  '23.236.66.13:50050',
  'NO_SOCKET'],
 'Memory': ['Memory Address'],
 'Other': ['8192*1024 bytes',
  'status:0',
  'UID:0',
  'GID:0',
  'PID:1514',
  'PID:1513',
  'Permission:0700',
  'PID:1515',
  'Permission:022',
  'PID:1516',
  'status:1',
  'PID:1518',
  'PID:1519',
  'PID:1520',
  'PID:1521',
  'PID:1522',
  'status:2',
  'PID:1524',
  'Sleep Duration',
  '